In [1]:
import os
from api import API_KEYS

os.environ["OPENAI_API_KEY"] = API_KEYS["OPENAI"]
os.environ["HUGGINGFACEHUB_API_TOKEN"] = API_KEYS["HUGGINFACE"]

## API CALLING TEST

### 프롬포트 생성

- PromptTemplate은 LLM에 문장을 전달하기 전에 문장 구성을 편리하게 만들어주는 역할
- LLM이 어떤 문장을 만들어야 하는지를 알려주는 역할을 수행

In [2]:
from langchain import PromptTemplate

template = "{product}를 홍보하기 위한 좋은 문구를 추천해줘?"

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

prompt.format(product="카메라")

'카메라를 홍보하기 위한 좋은 문구를 추천해줘?'

### OPENAI

In [3]:
from langchain.chat_models import ChatOpenAI

llm1 = ChatOpenAI(temperature=0,
                  model_name='gpt-4')

prompt = "상범이는 거북이를 키우고 있습니다. 상범이가 키우고 있는 동물은?"

print(llm1.predict(prompt))

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


거북이입니다.


In [4]:
from langchain_openai import ChatOpenAI

llm1 = ChatOpenAI(temperature=0,
                  model_name='gpt-4')

prompt = "상범이는 거북이를 키우고 있습니다. 상범이가 키우고 있는 동물은?"

print(llm1.invoke(prompt))

content='거북이입니다.' response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 42, 'total_tokens': 48}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-67cfc353-7d0c-40ba-981b-2d1d23d63fb8-0' usage_metadata={'input_tokens': 42, 'output_tokens': 6, 'total_tokens': 48}


### HUGGING FACE
- 시간마다 request할 수 있는 양의 한계가 있음
- google/flan-t5-xxl 모델 사용시, 바로 request 에러

In [7]:
from langchain import HuggingFaceHub

llm2 = HuggingFaceHub(repo_id = "google/flan-t5-small", 
                      model_kwargs={"temperature":0.8})

completion = llm2(prompt)
print(completion)

- Hugging Face Model은 제대로 대답을 하지 못함

### Compare

In [8]:
from langchain.model_laboratory import ModelLaboratory
model_lab = ModelLaboratory.from_llms([llm1, llm2])
model_lab.compare("대한민국의 가을은 몇 월부터 몇 월까지야?")

Input:
대한민국의 가을은 몇 월부터 몇 월까지야?

client=<openai.resources.chat.completions.Completions object at 0x10c612c50> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10c6283a0> model_name='gpt-4' temperature=0.0 openai_api_key=SecretStr('**********') openai_proxy=''


/opt/anaconda3/envs/llm/lib/python3.10/site-packages/pydantic/v1/main.py:996: RuntimeWarning: fields may not start with an underscore, ignoring "_input"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


대한민국의 가을은 일반적으로 9월부터 11월까지입니다.

HuggingFaceHub
Params: {'repo_id': 'google/flan-t5-small', 'task': 'text2text-generation', 'model_kwargs': {'temperature': 0.8}}
         



## Output Parser

In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0,
                 max_tokens=2048,
                 model_name='gpt-4',
                 )

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="7개의 팀을 보여줘 {subject}.format\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [10]:
query = "한국의 야구팀은?"

output = llm.predict(text=prompt.format(subject=query))

# 출력의 format 변경
parsed_results = output_parser.parse(output)
print(parsed_results)

['두산 베어스', '롯데 자이언츠', '삼성 라이온즈', 'SK 와이번스', 'KIA 타이거즈', 'LG 트윈스', 'NC 다이노스']


## PDF

In [11]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("The_Adventures_of_Tom_Sawyer.pdf")
document = loader.load()
document[5].page_content[:5000]

'Chapter 1    The Fence \n \nTom Sawyer lived with his aunt because his mother and \nfather were dead. Tom didn’t like going to school, and he didn’t like working. He liked playing and having adventures. One Friday, he didn’t go to school—he went to the river. \nAunt Polly was angry. “You’re a bad boy!” she said. \n“Tomorrow you can’t play with your friends because you didn’t go to school today. Tomorrow you’re going to work for me. You can paint the fence.” \nSaturday morning, Tom was not happy, but he started to \npaint the fence. His friend Jim was in the street. \nTom asked him, “Do you want to paint?” \nJim said, “No, I can’t. I’m going to get water.” \nThen Ben came to Tom’s house. He watched Tom and \nsaid, “I’m going to swim today. You can’t swim because you’re working.” \nTom said, “This isn’t work. I like painting.” \n“Can I paint, too?” Ben asked. \n“No, you can’t,” Tom answered. “Aunt Polly asked me \nbecause I’m a very good painter.” \nBen said, “I’m a good painter, too. P

In [12]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(document, embeddings)

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(temperature=0,
                 model_name = 'gpt-3.5-turbo',
                 )

retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

query = "마을 무덤에 있던 남자를 죽인 사람은 누구니?"
result = qa({'query': query})
print(result['result'])

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


마을 무덤에 있던 남자를 죽인 사람은 Injun Joe입니다.


## Lang Chain

- 여러개의 모델을 하나로 연결하는 것

### LLM Chain

In [14]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0,
                 model_name='gpt-4',
                 )

prompt = PromptTemplate(
    input_variables=['country'],
    template="{country}의 수도는 어디야?",
)

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("프랑스")

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'프랑스의 수도는 파리입니다.'

### Sequential Chain

In [17]:
prompt1 = PromptTemplate(
    input_variables=['sentence'],
    template = "다음 문장을 한글로 번역하세요. \n\n {sentence}"
)

chain1 = LLMChain(llm=llm, prompt=prompt1, output_key='translation')

prompt2 = PromptTemplate.from_template(
    "다음 문장을 네 문장으로 요약하세요. \n\n {sentence}"
)

chain2 = LLMChain(llm=llm, prompt=prompt2, output_key='summary')

In [19]:
from langchain.chains import SequentialChain

all_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=['sentence'],
    output_variables=['translation', 'summary'],
)

sentence="""
    The YOLOv10 paper introduces a new real-time end-to-end object detector that significantly advances the performance-efficiency boundary of the YOLO series. 
    The authors address the reliance on non-maximum suppression (NMS) for post-processing, which hinders end-to-end deployment and impacts inference latency. 
    They propose a consistent dual assignments strategy for NMS-free training, improving both performance and latency. Additionally, the paper introduces 
    a holistic efficiency-accuracy driven model design strategy, optimizing various components to reduce computational overhead and enhance capability. 
    Extensive experiments demonstrate that YOLOv10 achieves state-of-the-art performance and efficiency across various model scales, 
    outperforming previous models in terms of computation-accuracy trade-offs.
"""

ans = all_chain(sentence)

In [20]:
ans

{'sentence': '\n    The YOLOv10 paper introduces a new real-time end-to-end object detector that significantly advances the performance-efficiency boundary of the YOLO series. \n    The authors address the reliance on non-maximum suppression (NMS) for post-processing, which hinders end-to-end deployment and impacts inference latency. \n    They propose a consistent dual assignments strategy for NMS-free training, improving both performance and latency. Additionally, the paper introduces \n    a holistic efficiency-accuracy driven model design strategy, optimizing various components to reduce computational overhead and enhance capability. \n    Extensive experiments demonstrate that YOLOv10 achieves state-of-the-art performance and efficiency across various model scales, \n    outperforming previous models in terms of computation-accuracy trade-offs.\n',
 'translation': 'YOLOv10 논문은 YOLO 시리즈의 성능-효율성 경계를 크게 향상시키는 새로운 실시간 종단간 객체 탐지기를 소개합니다. \n저자들은 후처리를 위한 최대 비최대 억제(non-maximum suppression

In [24]:
print(ans['summary'])

The YOLOv10 paper presents a new real-time object detector that improves the performance-efficiency of the YOLO series. The authors tackle the issue of reliance on non-maximum suppression (NMS) which affects end-to-end deployment and inference latency. They suggest a dual assignments strategy for NMS-free training, enhancing both performance and latency. The paper also introduces a comprehensive model design strategy, optimizing different components to lower computational overhead and increase capability.


In [25]:
print(ans["translation"])

YOLOv10 논문은 YOLO 시리즈의 성능-효율성 경계를 크게 향상시키는 새로운 실시간 종단간 객체 탐지기를 소개합니다. 
저자들은 후처리를 위한 최대 비최대 억제(non-maximum suppression, NMS)에 대한 의존성을 다루며, 이는 종단간 배포를 방해하고 추론 지연에 영향을 미칩니다. 
그들은 NMS-free 훈련을 위한 일관된 이중 할당 전략을 제안하며, 이는 성능과 지연 시간 모두를 향상시킵니다. 또한, 이 논문은 다양한 구성 요소를 최적화하여 
계산 오버헤드를 줄이고 능력을 향상시키는 종합적인 효율성-정확도 중심의 모델 설계 전략을 소개합니다. 
다양한 모델 규모에서의 광범위한 실험은 YOLOv10이 최첨단 성능과 효율성을 달성하며, 계산-정확도 트레이드오프 측면에서 이전 모델들을 능가함을 보여줍니다.


## Memory

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain import ConversationChain

llm = ChatOpenAI(temperature=0,
                 model_name='gpt-4'
                 )

conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="상범이는 거북이입니다.")
conversation.predict(input="상범이는 1년 뒤에 30%의 확률로 암컷이 됩니다.")
conversation.predict(input="대호는 상범이를 세 마리 키웁니다.")
conversation.predict(input="대호가 키우는 상범이가 1년뒤에 모두 암컷일 확률은?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 상범이는 거북이입니다.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 상범이는 거북이입니다.
AI: 그렇군요, 상범이가 거북이라는 것을 알게 되어 반가워요. 거북이는 느린 움직임과 오래 살 수 있는 것으로 잘 알려져 있어요. 상범이는 어떤 종류의 거북이인가요? 그리고 상범이의 특별한 습성이나 특징이 있다면 알려주세요.
Human: 상범이는 1년 뒤에 30%의 확률로 암컷이 됩니다.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly convers

'상범이가 암컷이 될 확률이 30%라고 했을 때, 대호님이 키우는 세 마리의 상범이가 모두 암컷이 될 확률은 각각의 확률을 곱하는 것으로 계산할 수 있어요. 따라서 0.3 (30%) * 0.3 (30%) * 0.3 (30%) = 0.027, 즉 약 2.7%의 확률로 모두 암컷이 될 수 있습니다. 이는 매우 낮은 확률이지만, 불가능한 일은 아니에요.'

## Agent

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

llm = ChatOpenAI(temperature=0,
                 model_name='gpt-4'
                 )

tools = load_tools(['wikipedia', 'llm-math'], llm=llm)

agent = initialize_agent(tools,
                         llm,
                         agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         description='계산이 필요할 때 사용',
                         verbose=True
                         )

agent.run("아주대학교의 특징은? 2024년도 현재 올해 개교 후 몇 년?")

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
먼저 아주대학교의 특징에 대해 알아보기 위해 위키백과를 사용하겠습니다. 그리고 아주대학교의 개교 연도를 알아낸 후 2024년과의 차이를 계산하여 개교 후 몇 년이 되는지 알아보겠습니다.
Action: wikipedia
Action Input: 아주대학교
Observation: Page: Ajou University
Summary: Ajou University (Korean: 아주대학교; Hanja: 亞洲大學校; lit. Asia University) is a private research university located in Suwon, Gyeonggi Province, South Korea. It was founded in 1973 by Park Changwon.
Ajou University started as Ajou Engineering Junior College with the aim of exchanging technology and culture through education between France and South Korea.
Ajou is located on a single campus witfh all details, research-oriented university hospitals, Techno Valley, Samsung Electronics headquarters, CJ Blossom Park, administrative town (Gyeonggi Provincial Office, Gyeonggi Provincial Council, Education Office, Gyeonggi Representative Library), and legal town (Suwon Court, Suwon Prosecutor's Office). Gyeonggi-do Pharmaceutical Association has signed MOUs with Ajou.
Its busine

'아주대학교는 1973년에 설립되었으며, 프랑스와 한국 간의 교육을 통한 기술과 문화 교류를 목표로 시작한 곳입니다. 이는 한 캠퍼스에 위치해 있으며, 연구 중심의 대학 병원, 테크노 밸리, 삼성전자 본사, CJ 블로썸 파크, 행정 도시(경기도청, 경기도의회, 교육청, 경기대표 도서관), 법률 도시(수원 법원, 수원 검찰청) 등이 있습니다. 또한, 경기도 약사회와 MOU를 체결하였습니다. 그리고 그들의 사업 대학은 한국 교육부의 세계 수준의 연구 중심 대학 프로젝트의 일환으로 금융 공학 부서를 개설할 권한을 받은 한국에서 유일한 곳이며, AACSB와 EFMD 인증 기관의 회원입니다. 2024년에는 개교 51주년을 맞이하게 됩니다.'

## RAG Chatbot

In [2]:
from langchain.document_loaders import TextLoader

documents = TextLoader("AI.txt").load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)

In [4]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name = 'all-MiniLM-L6-v2')

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/opt/anaconda3/envs/llm/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a 

In [5]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

In [6]:
from langchain.chat_models import ChatOpenAI

model_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(model_name = model_name)

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type='stuff', verbose=True)

query = 'AI란?'
matching_docs = db.similarity_search(query)
ans = chain.run(input_documents=matching_docs, question=query)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Artificial intelligence (AI) is the intelligence of machines or software, as opposed to the intelligence of humans or animals. It is a field of study in computer science that develops and studies intelligent machines. Such machines may be called AIs.

AI technology is widely used throughout industry, government, and science. Some high-profile applications are: advanced web search engines (e.g., Google Search), recommendation systems (used by YouTube, Amazon, and Netflix), understanding human speech (such as Google Assistant, Siri, and Alexa), self-driving cars (e.g., Waymo), generative and creative tools (ChatGPT and AI art), and superhuman play and analysis in strategy games (such as chess and Go).[

In [12]:
print(ans.replace('.', '.\n'))

AI(인공지능)는 인간이나 동물의 지능과는 달리 기계나 소프트웨어의 지능을 의미합니다.
 컴퓨터과학의 한 분야로, 지능적인 기계를 개발하고 연구하는 분야입니다.
 AI 기술은 산업, 정부, 과학 분야에서 널리 사용되며, 구글 검색 엔진, YouTube, Amazon, Netflix의 추천 시스템, 구글 어시스턴트, Siri, Alexa와 같은 음성인식 기술, 웨이모와 같은 자율주행 자동차, ChatGPT 및 AI 아트와 같은 창의적인 도구, 그리고 체스와 바둑과 같은 전략 게임에서의 초인간적인 플레이와 분석 등이 있습니다.

